In [21]:
import qiskit
from qiskit.quantum_info import state_fidelity
import numpy as np
from scipy.sparse import linalg as LA
import qib
import matplotlib.pyplot as plt
import rqcopt as oc
import scipy
import scipy.sparse as sp
from qiskit.circuit.library import CXGate, CYGate, CZGate
from qiskit import Aer, execute, transpile
from qiskit.providers.aer.noise import NoiseModel, errors
from qiskit.converters import circuit_to_dag

import sys
sys.path.append("../../src/qetu")
from qetu import qetu_rqc_oneLayer


def construct_ising_local_term(J, g, ndim, h=0):
    X = np.array([[0.,  1.], [1.,  0.]])
    Z = np.array([[1.,  0.], [0., -1.]])
    I = np.identity(2)
    return J*np.kron(Z, Z) + g*(0.5/ndim)*(np.kron(X, I) + np.kron(I, X)) + h*(0.5/ndim)*(np.kron(Z, I) + np.kron(I, Z))


In [16]:

def controlled_trotter(c1, c2, L, Lx, Ly, J, g, dag=False, nsteps=1, multi_ancilla=1):
    if c1/(2*nsteps) > 0.1:
        nsteps = int(np.ceil(c1/(2 * 0.1)))
        print("nsteps set to: ", nsteps)
        print("dt set to: ",  c1/(2*nsteps))
    t = c1/(2*nsteps)

    # permutations specifying gate layout
    # Verticals
    perms_vercs = []
    for i in range(Ly):
        start_ind = Lx*i
        perms_verc = []
        for j in range(start_ind, start_ind+Lx):
            perms_verc += [j, (j+Lx)%L]
        perms_vercs.append(perms_verc)
    if Ly == 4:
        perms_vercs = [perms_vercs[0]+perms_vercs[2], perms_vercs[1]+perms_vercs[3]]

    # Horizontals
    perms_horzs = []
    for i in range(Lx):
        start_ind = i
        perms_horz = []
        for j in range(start_ind, L, Lx):
            if start_ind != Lx-1:
                perms_horz += [j, j+1]
            else:
                perms_horz += [j, j+1-Lx]
        perms_horzs.append(perms_horz)
    if Lx == 4:
        perms_horzs = [perms_horzs[0]+perms_horzs[2], perms_horzs[1]+perms_horzs[3]]
    perm_set = perms_vercs + perms_horzs
    #print(perm_set)
    method_start = oc.SplittingMethod.suzuki(len(perm_set), 1)
    indices = method_start.indices
    coeffs = method_start.coeffs
    hloc = construct_ising_local_term(J, g, 2)

    K = []
    for perm in perm_set:
        K_layer = [None for _ in range(L)]
        for j in range(len(perm)//2):
            K_layer[perm[2*j]] = CYGate
            K_layer[perm[2*j+1]] = CZGate
        K.append(K_layer)

    Vlist_start = []
    perms = []
    Ks = []
    for i, c in zip(indices, coeffs):
        Vlist_start.append(scipy.linalg.expm(-1j*c*t*hloc))
        perms.append(perm_set[i])
        Ks.append(K[i])
    Vlist_gates = []
    for V in Vlist_start:
        qc2 = qiskit.QuantumCircuit(2)
        qc2.unitary(V, [0, 1], label='str')
        Vlist_gates.append(qc2)

    qc = qiskit.QuantumCircuit(L+multi_ancilla)
    if dag:
        for anc_ind in range(multi_ancilla):
            qc.x(L+anc_ind)

    for layer, qc_gate in enumerate(Vlist_gates):

        for anc_ind in range(multi_ancilla):
            qc.x(L+anc_ind)
        controlled_ops = 0
        for j in range(L):
            if Ks[layer][j]:
                control = controlled_ops % multi_ancilla
                qc.append(Ks[layer][j](), [L, L-1-j])
                controlled_ops += 1
        for anc_ind in range(multi_ancilla):
            qc.x(L+anc_ind)

        for n in range(nsteps):
            for j in range(len(perms[layer])//2):
                qc.append(qc_gate.to_gate(), [L-(perms[layer][2*j]+1), L-(perms[layer][2*j+1]+1)])

        for anc_ind in range(multi_ancilla):
            qc.x(L+anc_ind)
        controlled_ops = 0
        for j in range(L):
            if Ks[layer][j]:
                control = controlled_ops % multi_ancilla
                qc.append(Ks[layer][j](), [L+control, L-1-j])
                controlled_ops += 1
        for anc_ind in range(multi_ancilla):
            qc.x(L+anc_ind)
    qc.cp(-c2, L, 0)
    qc.x(0)
    qc.cp(-c2, L, 0)
    qc.x(0)

    if dag:
        for anc_ind in range(multi_ancilla):
            qc.x(L+anc_ind)
    return qc



def run_qetu(Lx, Ly, g, c1, c2, mu=0.95, d=12, M=2, return_circ=False, qc_init=None, 
             full_measure=False, return_one_layer=False, a_assess=None, reverse=False, 
             return_poly=False):

    qc_cU = controlled_trotter(c1, c2, L, Lx, Ly, J, g, dag=False)
    qc_cU_dag = controlled_trotter(c1, c2, L, Lx, Ly, J, g, dag=True)
    qc_cfUs = [qc_cU, qc_cU_dag]
    
    ket_0 = np.array([1,0])
    
    # QETU Circuit:
    c, phis_max_iter, = (0.95, 10)
    qc_qetu, _ = qetu_rqc_oneLayer(L, 0, 0, 1, mu, d=d, c2=0,
                                    qc_cU_custom=(qc_cfUs[0], qc_cfUs[1]),
                                       a_assess=a_assess, reverse=reverse
                                    )
    if return_one_layer:
        return qc_qetu
    
    qcs_qetu = []
    for m in range(M):
        qcs_qetu.append(qc_qetu)

    if return_circ:
        mid_cbits = len(qcs_qetu)
        qc = qiskit.QuantumCircuit(L+1, (L+1 if full_measure else 1) + mid_cbits)
        if qc_init is not None:
            qc.append(qc_init.to_gate(), [i for i in range(L)])
        
        for j, qc_qetu in enumerate(qcs_qetu):
            qc.append(qc_qetu.to_gate(), [i for i in range(L+1)])
            qc.measure(L, j)
        return qc, mid_cbits
    else:
        backend = Aer.get_backend("statevector_simulator")
        qc_RQC = qiskit.QuantumCircuit(L+1, L+1)
        qc_RQC.initialize(np.kron(ket_0, state))
        if qc_init is not None:
            qc_RQC.append(qc_init.to_gate(), [i for i in range(L)])
        
        for qc_qetu in qcs_qetu:
            qc_RQC.append(qc_qetu.to_gate(), [i for i in range(L+1)])
            bR = execute(transpile(qc_RQC), backend).result().get_statevector().data
            aR = sp.kron(np.array([[1,0],[0,0]]), sp.eye(2**L), format='csr') @ bR
            print("Success Prob: ", np.linalg.norm(aR)**2)
            aR = aR / np.linalg.norm(aR)
            qc_RQC.reset([i for i in range(L+1)])
            qc_RQC.initialize(aR)
        
        toPlot = [state_fidelity(aR[:2**L], eigenvectors_sort[:, i]) for i in range(10)]
        plt.plot([i for i in range(len(toPlot))], toPlot)
        plt.scatter([i for i in range(len(toPlot))], toPlot)
        #plt.show()
        print("State Fid: ", toPlot[0])
    
        mid_cbits = len(qcs_qetu)
        qc = qiskit.QuantumCircuit(L+1, 1 + mid_cbits)
        qc.initialize(np.kron(ket_0, state))
        if qc_init is not None:
            qc.append(qc_init.to_gate(), [i for i in range(L)])
        for j, qc_qetu in enumerate(qcs_qetu):
            qc.append(qc_qetu.to_gate(), [i for i in range(L+1)])
            qc.measure(L, j)
            
        noise_model = NoiseModel()
        tr = transpile(qc, basis_gates=noise_model.basis_gates+['unitary', 'initialize', 'cx'])
        dag = circuit_to_dag(tr)
        count_ops = dag.count_ops()

        counts_q = {}
        for i in range(L+1):
            counts_q[i] = {"s": 0, "d": 0}
        
        for gate in tr.data:
            if len(gate.qubits) == 1:
                counts_q[gate.qubits[0].index]["s"] += 1
            elif len(gate.qubits) == 2:
                counts_q[gate.qubits[0].index]["d"] += 1
                counts_q[gate.qubits[1].index]["d"] += 1
        counts_q["gates"] = count_ops['unitary'] + count_ops['cx']
        
        if return_poly:
            return toPlot, counts_q, aR[:2**L], phis_[2]
        return toPlot, counts_q, aR[:2**L]

In [53]:
"""
    Adiabatic Evolution Implementation.
"""
def trotter(Lx, Ly, tau, L, J_i, h_i, g_i, J_f, h_f, g_f, lamb):
    L = Lx * Ly
    assert lamb <= 1 and lamb >= 0
    J = lamb*J_f + (1-lamb)*J_i
    g = lamb*g_f + (1-lamb)*g_i
    h = lamb*h_f + (1-lamb)*h_i

    qc = qiskit.QuantumCircuit(L)
    hloc = construct_ising_local_term(J, g, 2, h=h)
        
    # permutations specifying gate layout
    # Verticals
    perms_vercs = []
    for i in range(Ly):
        start_ind = Lx*i
        perms_verc = []
        for j in range(start_ind, start_ind+Lx):
            perms_verc += [j, (j+Lx)%L]
        perms_vercs.append(perms_verc)
    if Ly == 4:
        perms_vercs = [perms_vercs[0]+perms_vercs[2], perms_vercs[1]+perms_vercs[3]]
    # Horizontals
    perms_horzs = []
    for i in range(Lx):
        start_ind = i
        perms_horz = []
        for j in range(start_ind, L, Lx):
            if start_ind != Lx-1:
                perms_horz += [j, j+1]
            else:
                perms_horz += [j, j+1-Lx]
        perms_horzs.append(perms_horz)
    if Lx == 4:
        perms_horzs = [perms_horzs[0]+perms_horzs[2], perms_horzs[1]+perms_horzs[3]]
    perm_set = perms_vercs + perms_horzs
    perms = perm_set

    
    method_start = oc.SplittingMethod.suzuki(len(perm_set), 1)
    indices = method_start.indices
    coeffs = method_start.coeffs

    # unitaries used as starting point for optimization
    Vlist_start = []
    perms = []
    for i, c in zip(indices, coeffs):
        Vlist_start.append(scipy.linalg.expm(-1j*c*tau*hloc))
        perms.append(perm_set[i])
    Vlist_gates = []
    for V in Vlist_start:
        qc2 = qiskit.QuantumCircuit(2)
        qc2.unitary(V, [0, 1], label='str')
        Vlist_gates.append(qc2)
    
    for layer, qc_gate in enumerate(Vlist_gates):     
        for j in range(len(perms[layer])//2):
            qc.append(qc_gate.to_gate(), [L-(perms[layer][2*j]+1), L-(perms[layer][2*j+1]+1)])
    return qc


def run_adiabatic(Lx, Ly, g, T, S, state, return_state=False, h_i=0, h_f=0, J=-1):
    L = Lx*Ly
    tau = 1/S
    t_s = np.linspace(0, T, S*T)
    sch = lambda t, T: np.sin(np.pi*t/(2*T))**2
    
    qc = qiskit.QuantumCircuit(L)
    for s in range(S*T):
        qc.append(trotter(Lx, Ly, tau, L, 0, h_i, g, J, h_f, g, sch(t_s[s], T)).to_gate(), [i for i in range(L)])
    
    backend = Aer.get_backend("statevector_simulator")
    qc_ = qiskit.QuantumCircuit(L)
    qc_.initialize(state)
    qc_.append(qc.to_gate(), [i for i in range(L)])
    final = execute(transpile(qc_), backend).result().get_statevector().data


    if return_state:
        return qc
    else:
        noise_model = NoiseModel()
        dag = circuit_to_dag(transpile(qc_, basis_gates=noise_model.basis_gates+['unitary', 'initialize', 'cx']))
        count_ops = dag.count_ops()
        
        return [state_fidelity(final, eigenvectors_sort[:, i]) for i in range(10)], {"gates": count_ops['unitary']}, final


In [ ]:
"""
    Perform LB scan for TFIM 2D.
"""


sps_LB = {}
for Lx, Ly in [(5,4)]:
    sps_LB[(Lx, Ly)] = {}

    J, g, h = (-1, 1, 0)
    L = Lx * Ly
    
    # construct Hamiltonian
    latt = qib.lattice.IntegerLattice((Lx, Ly), pbc=True)
    field = qib.field.Field(qib.field.ParticleType.QUBIT, latt)
    
    H_i = qib.IsingHamiltonian(field, 0, 0., g).as_matrix()
    eigenvalues, eigenvectors = LA.eigsh(H_i, k=10)
    idx = eigenvalues.argsort()
    eigenvectors_sort = eigenvectors[:,idx]
    state = eigenvectors_sort[:, 0]
    #state = np.array(eigenvectors_sort[:, 0].reshape(-1))[0]
    
    hamil = qib.IsingHamiltonian(field, J, h, g).as_matrix()
    eigenvalues, eigenvectors = LA.eigsh(hamil, k=10)
    idx = eigenvalues.argsort()
    eigenvalues_sort = eigenvalues[idx]
    eigenvectors_sort = eigenvectors[:,idx]
    ground_state = eigenvectors_sort[:, 0]

    qc_A = run_adiabatic(Lx, Ly, g, 8, 3, state, return_state=True)

    LBs = range(-40, -34, 1)
    for LB in LBs:
        spectrum_upper_bound = 0 
        spectrum_lower_bound = LB
        max_spectrum_length = spectrum_upper_bound - spectrum_lower_bound
        c1 = (np.pi) / (max_spectrum_length)
        c2 = - c1 * spectrum_lower_bound
    
        fid, gates, final = run_qetu(Lx, Ly, g, c1, c2, mu=0.98, d=12, M=1, qc_init=qc_A)
        print("sp: ", fid[0])
        sps_LB[(Lx, Ly)][LB] = fid[0]


In [ ]:
"""
    Perform UB scan for TFIM 2D.
"""


sps_UB = {}
for Lx, Ly in [(3,3)]:
    sps_UB[(Lx, Ly)] = {}

    J, g, h = (-1, 1, 0)
    L = Lx * Ly
    
    # construct Hamiltonian
    latt = qib.lattice.IntegerLattice((Lx, Ly), pbc=True)
    field = qib.field.Field(qib.field.ParticleType.QUBIT, latt)
    
    H_i = qib.IsingHamiltonian(field, 0, 0., g).as_matrix()
    eigenvalues, eigenvectors = LA.eigsh(H_i, k=10)
    idx = eigenvalues.argsort()
    eigenvectors_sort = eigenvectors[:,idx]
    state = eigenvectors_sort[:, 0]
    #state = np.array(eigenvectors_sort[:, 0].reshape(-1))[0]
    
    
    hamil = qib.IsingHamiltonian(field, J, h, g).as_matrix()
    eigenvalues, eigenvectors = LA.eigsh(hamil, k=10)
    idx = eigenvalues.argsort()
    eigenvalues_sort = eigenvalues[idx]
    eigenvectors_sort = eigenvectors[:,idx]
    ground_state = eigenvectors_sort[:, 0]

    qc_A = run_adiabatic(Lx, Ly, g, 3, 3, state, return_state=True, h_i=-0, h_f=0)


    UBs = range(-10, -40, -5)
    for UB in UBs:
        spectrum_upper_bound = UB 
        spectrum_lower_bound = -39
        max_spectrum_length = spectrum_upper_bound - spectrum_lower_bound
        c1 = (np.pi) / (max_spectrum_length)
        c2 = - c1 * spectrum_lower_bound
    
        fid, gates, final = run_qetu(Lx, Ly, g, c1, c2, mu=0.98, d=12, M=1, qc_init=qc_A)
        print("sp: ", fid[0])
        sps_UB[(Lx, Ly)][UB] = fid[0]


In [70]:
#fids = {}
#fids_A = {}
#cxs = {}
#cxs_A = {}
"""bounds = {(3,3): (-19, -13),
          (4,4): (-38, -24),
          (6,3): (-40, -30),
          (5, 4): (-43, -35),
          (5, 5): (-50, -40)
         }"""

for Lx, Ly in [(3, 3), (4, 4), (5, 4)]:
    #fids[(Lx, Ly)] = []
    #cxs[(Lx, Ly)] = []
    #fids_A[(Lx, Ly)] = []
    #cxs_A[(Lx, Ly)] = []

    J, g, h = (-1, 1, 0)
    L = Lx * Ly
    latt = qib.lattice.IntegerLattice((Lx, Ly), pbc=True)
    field = qib.field.Field(qib.field.ParticleType.QUBIT, latt)
    
    H_i = qib.IsingHamiltonian(field, 0, 0., g).as_matrix()
    eigenvalues, eigenvectors = LA.eigsh(H_i, k=10)
    idx = eigenvalues.argsort()
    eigenvectors_sort = eigenvectors[:,idx]
    state = eigenvectors_sort[:, 0]
    
    hamil = qib.IsingHamiltonian(field, J, h, g).as_matrix()
    eigenvalues, eigenvectors = LA.eigsh(hamil, k=10)
    idx = eigenvalues.argsort()
    eigenvalues_sort = eigenvalues[idx]
    eigenvectors_sort = eigenvectors[:,idx]
    ground_state = eigenvectors_sort[:, 0]
    print("Ground State Energy", eigenvalues_sort[0].real)
    print("First Excited State Energy", eigenvalues_sort[1].real)
    print("Second Excited State Energy", eigenvalues_sort[2].real)
    print("\n Spectral Gap:  \n", eigenvalues_sort[2].real - eigenvalues_sort[0].real)

    """spectrum_upper_bound = bounds[(Lx, Ly)][1]
    spectrum_lower_bound = bounds[(Lx, Ly)][0]
    max_spectrum_length = spectrum_upper_bound - spectrum_lower_bound
    c1 = (np.pi) / (max_spectrum_length)
    c2 = - c1 * spectrum_lower_bound
    eigenvalues_tr = eigenvalues_sort * c1 + c2
    a_values = np.array([np.cos(eig/2) for eig in eigenvalues_tr])
    
    for T in range(2, 15, 2):
        qc_A = run_adiabatic(Lx, Ly, g, T, 3, state, return_state=True, h_i=-0, h_f=0)
        fid, gates, final = run_qetu(Lx, Ly, g, c1, c2, mu=0.8, d=4, M=1, qc_init=qc_A, a_assess=2)
        fids[(Lx, Ly)].append(fid[0])
        cxs[(Lx, Ly)].append(gates['gates'])

    for T in range(4, 18, 2):
        fid, gates, _ = run_adiabatic(Lx, Ly, g, T, 3, state, h_i=-0, h_f=0)
    
        fids_A[(Lx, Ly)].append(fid[0])
        cxs_A[(Lx, Ly)].append(gates['gates'])"""

Ground State Energy -19.131366809073914
First Excited State Energy -19.130864649143565
Second Excited State Energy -12.035647677612857

 Spectral Gap:  
 7.095719131461056
Ground State Energy -34.01059755084611
First Excited State Energy -34.01059699946813
Second Excited State Energy -26.725061975707234

 Spectral Gap:  
 7.285535575138873
Ground State Energy -42.51324373020311
First Excited State Energy -42.51324371855706
Second Excited State Energy 35.25818667407672

 Spectral Gap:  
 77.77143040427983


In [ ]:
plt.subplots_adjust(left=0.2, bottom=0.1, right=0.9, top=0.8, wspace=0.5, hspace=0.2)
plt.rcParams["figure.figsize"] = (12, 10)
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 150

colors = ['red', 'blue', 'green', 'orange']

l = 0
for Lx, Ly in [(3, 3), (4,4), (6, 3), (5, 4)]:
    plt.plot(cxs[(Lx, Ly)], [1-fid for fid in fids[(Lx, Ly)]], label=f'Adiabatic+QETU, {Lx}x{Ly} lattice', color=colors[l],linewidth=3.0)  
    plt.scatter(cxs[(Lx, Ly)], [1-fid for fid in fids[(Lx, Ly)]], color=colors[l], s=100)
    
    plt.plot(cxs_A[(Lx, Ly)], [1-fid for fid in fids_A[(Lx, Ly)]], label=f'solo-Adiabatic, {Lx}x{Ly} lattice', ls='--', color=colors[l],linewidth=3.0)  
    plt.scatter(cxs_A[(Lx, Ly)], [1-fid for fid in fids_A[(Lx, Ly)]], color=colors[l], s=100)    
    l += 1
plt.yscale('log')
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.xlabel('Total Number of Two Qubit Gates', size=30)
plt.ylabel(r'$1-|<\psi|\psi_f>|$', size=30)
plt.legend(prop={'size': 18})

#plt.savefig('./figs/TFIM_2D_lattices_infid.png')
#plt.savefig('./figs/TFIM_2D_lattices_infid.pdf')


In [63]:
print(fids, cxs, '\n')
print(fids_A, cxs_A)

{(5, 4): [0.047000684524338636, 0.35429065641629226, 0.6647158154320717, 0.8401787503187553, 0.9256500397632285, 0.9615722953944339, 0.9787371877167061], (4, 4): [0.39742359609989375, 0.9569440154811948, 0.9811463005590868, 0.9878299216757925, 0.9917465525240409, 0.9935850822334509, 0.9950643798029734], (6, 3): [0.10440177717953354, 0.817027256557907, 0.9681197785291125, 0.9914564060427209, 0.9944309852343269, 0.9965204696052431, 0.997362893216656], (3, 3): [0.7211514551544113, 0.9849005036455258, 0.9970299896597282, 0.9981561023153802, 0.9987621041352132, 0.9990373840779736, 0.9991835919841237]} {(5, 4): [1376, 1832, 2288, 2744, 3200, 3656, 4112], (4, 4): [1016, 1352, 1688, 2024, 2360, 2696, 3032], (6, 3): [1242, 1656, 2070, 2484, 2898, 3312, 3726], (3, 3): [668, 866, 1064, 1262, 1460, 1658, 1856]} 

{(5, 4): [0.15180289926474788, 0.2898967882941344, 0.39763639203173234, 0.5040437931510187, 0.5797187222540855, 0.6495748563036126, 0.703512872619141], (4, 4): [0.2614658652860284, 0.4221